# Chicago Taxi Rides Analysis

In this project, I will analyze taxi ride data, explore patterns, and draw meaningful conclusions. The data I will work with includes information as results from SQL queries from databases that has information on competitor taxi companies, the number of rides, the neighborhoods where rides ended, and the weather at the time the ride started.

The project involves several steps:

Data Retrieval: I will start by retrieving data from resulting CSV files, which contain information about the number of rides for each taxi company and the average number of rides that ended in each neighborhood. Another CSV file with weather information on the hour is used.

Exploratory Data Analysis: I will study the data, ensure the data types are correct, and identify key patterns and trends. This will involve creating visualizations such as graphs to better understand the data.

Hypothesis Testing: I will test a hypothesis that “The average duration of rides from the Loop to O’Hare International Airport changes on rainy Saturdays.” This will involve statistical analysis and interpretation of the results.

Conclusion: Finally, I will draw overall conclusions based on your analysis and findings.

# Initialization

In [ ]:
# Load the Libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Ignore filter warnings that asks for .copy()
warnings.filterwarnings('ignore')

### Load the Data

In [ ]:
# Read the data from .csv files

try:
    company = pd.read_csv('./datasets/moved_project_sql_result_01.csv')
    neighborhood = pd.read_csv('./datasets/moved_project_sql_result_04.csv')
    loop_airport = pd.read_csv('./datasets/moved_project_sql_result_07.csv')
except:
    company = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/learning-materials/data-analyst-eng/moved_project_sql_result_01.csv')
    neighborhood = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/learning-materials/data-analyst-eng/moved_project_sql_result_04.csv')
    loop_airport = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/learning-materials/data-analyst-eng/moved_project_sql_result_07.csv')



### Prepare the Data

In [ ]:
# Show information on the dataframes and their first five rows.
company.info()
display(company.head())
neighborhood.info()
display(neighborhood.head())
loop_airport.info()
display(loop_airport.head())

### Convert data types into correct ones

In [ ]:
# Convert 'duration_seconds' into int
if np.array_equal(loop_airport['duration_seconds'], loop_airport['duration_seconds'].astype('int')):
  loop_airport['duration_seconds'] = loop_airport['duration_seconds'].astype('int')

# Convert 'start_ts' into datetime format
loop_airport['start_ts'] = pd.to_datetime(loop_airport['start_ts'])

loop_airport.info()

### Check for duplicates

In [ ]:
# Count the duplicates
print("Duplicates")
print("Company dataframe:", company.duplicated().sum())
print("Neighborhood dataframe:", neighborhood.duplicated().sum())
print("Loop to O'Hare dataframe:", loop_airport.duplicated().sum())

There are 197 duplicated rows in the loop_airport dataframe. The records are rounded to the nearest hour, but they may actually be distinct records. It would be better to keep these duplicates instead of removing them from the dataframe.

### Handle missing values
Looking at the dataframes' info() output, there were no missing values.

# Analyze the data

In [ ]:
# Identify the top 10 neighborhoods in terms of drop-offs
top10_neighborhoods = neighborhood.nlargest(10, 'average_trips')
display(top10_neighborhoods)

In [ ]:
# Make a graph for taxi companies and number of rides
sorted_company = company.sort_values('trips_amount', ascending=True)
fig, ax = plt.subplots(figsize=(8,13))
ax.barh(sorted_company['company_name'], sorted_company['trips_amount'])
ax.set_title('Taxi companies and number of rides')
ax.set_xlabel('Number of Rides')
ax.set_ylabel('Company Name')
ax.set_ylim(-0.5, len(sorted_company)-0.5)

# Add a dotted grid
ax.grid(True, which='both', axis='x', linestyle=':', color='black', alpha=0.6)

plt.show()


Based on this graph, it can be said that:

- Flash Cab is the most popular taxi company, with close to 20,000 rides. This suggests that Flash Cab might have a larger fleet, better service, or more competitive pricing compared to other companies.

- The number of rides decreases significantly after Flash Cab. The second most popular company, Taxi Affiliation Service Yellow, has a little more than half the number of rides compared to Flash Cab.

- Most of the taxi companies have a similar number of rides, indicating that they have a comparable market share. However, none of them come close to Flash Cab in terms of the number of rides.

- There is a wide range in the number of rides among the taxi companies, indicating a significant variation in their market shares. This could be due to various factors such as the size of the taxi company, the areas they serve, their pricing, and the quality of their service.

In [ ]:
# Make a graph for the top 10 neighborhoods by number of dropoffs
fig, ax = plt.subplots(figsize=(15,6))
ax.barh(top10_neighborhoods['dropoff_location_name'], top10_neighborhoods['average_trips'])
ax.set_title('Top 10 neighborhoods by number of dropoffs')
ax.set_xlabel('Neighborhood')
ax.set_ylabel('Average Number of Trips')
plt.show()

Based on this graph, it can be said that:

- Loop is the most popular drop-off neighborhood, with over 10,000 trips. Since the Loop is located in the heart of downtown Chicago, it's busy area with many attractions, businesses, or residential buildings.

- The second and third most popular drop-off neighborhoods are River North and Streeterville, respectively.

- There is a noticeable decrease in the number of trips from West Loop to O'Hare, and then the numbers gradually decrease across other neighborhoods. This could indicate that these areas are less populated or less frequented by taxi passengers. Other modes of transportation could also be a factor.

- The least popular neighborhood among the top 10 is Sheffield & DePaul, with the least number of drop-offs.


### Test the hypothesis:
"The average duration of rides from the Loop to O'Hare International Airport changes on rainy Saturdays."

- Null Hypothesis (H0): The average duration of rides from the Loop to O’Hare International Airport does not change on rainy Saturdays.
- Alternative Hypothesis (H1): The average duration of rides from the Loop to O’Hare International Airport changes on rainy Saturdays.

We will use a t-test to test these hypotheses. The t-test is used to determine if there is a significant difference between the means of two groups. In this case, the two groups are the durations of rides on rainy Saturdays and the durations of rides on other days.

In [ ]:
# Create a 'day_of_week' column
loop_airport['day_of_week'] = loop_airport['start_ts'].dt.dayofweek

# Filter data for Saturdays
df_saturday = loop_airport[loop_airport['day_of_week'] == 5]

# Further divide the data into rainy and non-rainy days
df_rainy = df_saturday[df_saturday['weather_conditions'] == 'Bad']
df_non_rainy = df_saturday[df_saturday['weather_conditions'] == 'Good']

# Perform t-test
result = stats.ttest_ind(df_rainy['duration_seconds'], df_non_rainy['duration_seconds'])

alpha = 0.05

print(result.pvalue)

if result.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

Since the pvalue is significantly less than the alpha, we can reject the null hypothesis. The averate duration of rides from the Loop to O'Hare International Airport changes on rainy Saturdays. This could be because when it rains, people try to be careful and drive slower and possibly create traffic, therefore changing the duration of rides.

# Conclusion

In this project, I performed an extensive analysis of taxi rides in Chicago. I started by importing and studying data from three CSV files, two of which contained information about the number of rides for each taxi company and the average number of rides that ended in each neighborhood.

The exploratory data analysis revealed that Flash Cab was the most popular taxi company, with a significantly higher number of rides compared to other companies. This suggests that Flash Cab might have a larger fleet, better service, or more competitive pricing.

I also identified the top 10 neighborhoods in terms of drop-offs. The Loop neighborhood had the highest number of drop-offs, which makes sense since it's located in downtown Chicago, which is an area with many attractions, businesses, or residential buildings.

Next, I tested the hypothesis that “The average duration of rides from the Loop to O’Hare International Airport changes on rainy Saturdays.” The p-value from the t-test was extremely small (7.397770692813658e-08), leading me to reject the null hypothesis. This suggests that the weather conditions do have an impact on the ride duration.

In conclusion, this project provided valuable insights into the taxi usage patterns in Chicago and highlighted the influence of various factors such as the taxi company, drop-off neighborhood, and weather conditions on these patterns. These insights are useful for Zuber's as it plans on launching its platform.